# Testing likelihood runtimes


The rate limiting step for linkelihood based inference is the time taken to obtain a single likelihood sample.

We want this to be as fast as possible 

See e.g. tips on using numba: https://numba.pydata.org/numba-doc/latest/user/performance-tips.html

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import json 
import pandas as pd 
try:
    sys.path.remove("../py_src") # Hacky way to add higher directory to python modules path. 
except:
    pass
sys.path.append("../py_src") # Means that I dont have to make src/ a proper python package

In [3]:
from system_parameters import SystemParameters
from pulsars import Pulsars
from synthetic_data import SyntheticData
from model import LinearModel
from kalman_filter import KalmanFilter
from priors import priors_dict,bilby_priors_dict
import numpy as np 
import copy 
import logging 
logger = logging.getLogger().setLevel(logging.ERROR)

#Function to create synthetic data
def create_synthetic_data(h,measurement_noise,seed):

    P    = SystemParameters(h=h,σp=None,σm=measurement_noise,seed=seed)       #define the system parameters as a class
    PTA  = Pulsars(P)               # setup the PTA
    data = SyntheticData(PTA,P)     # generate some synthetic data

    return P,PTA,data

#Fit model to data and return likelihood
def setup_kalman_likelihood(data,P,PTA,measurement_model):

    _P = copy.copy(P)
    _PTA = copy.copy(PTA)
    _P.measurement_model = measurement_model
    

    #Define the model 
    model = LinearModel(_P)

    #Initialise the Kalman filter
    KF = KalmanFilter(model,data.f_measured,_PTA)

    #Create a bilby dict 
    init_p,parameter_priors = bilby_priors_dict(_PTA,_P)
    
    sampled_parameters = parameter_priors.sample(1)
    return KF,sampled_parameters

### Create some data



In [4]:
h = 1e-12
measurement_noise = 1e-11
seed=1236
P,PTA,data = create_synthetic_data(h,measurement_noise,seed)

In [5]:
measurement_model = "earth"
KF,parameters_sample = setup_kalman_likelihood(data,P,PTA,measurement_model)


### Now time the function

In [6]:
def function_to_time(KF,params):
    KF.likelihood(params)

In [8]:
%timeit function_to_time(KF,parameters_sample)

4.78 ms ± 34.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
